In [10]:
import csv
from pprint import pprint
from collections import defaultdict
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_table("anime.csv", sep=",")
us = pd.read_table("rating.csv", sep=",")

In [3]:
animeList={} #Anime and their data
genreList=defaultdict(list) #Animes sorted by genre
userList=defaultdict(list) #User's list of anime and rating in a (anime_id,rating) structure
userRating=defaultdict(list) #User's rating
userCategory=defaultdict(set) #User's list of categories they have watched

In [4]:
# To Get data from animeList. dictionary = animeList['id']['data']
def getData(dictionary):
    if not dictionary.isnull().values[0]:
        return dictionary.values[0]
    else:
        return 0

In [5]:
def getCategory(data):
    cat=[]
    if data:
        for x in data.split(', '):
            cat.append(x)
    return cat

In [6]:
for index, row in df.iterrows():
    animeList[row['anime_id']]=df.loc[[index]]
    if not df.loc[[index]]['genre'].isnull().values[0]:
        for x in row['genre'].split(', '):
            genreList[x].append(row['anime_id'])

In [7]:
for index, row in us.iterrows():
    userList[row['user_id']].append(row['anime_id'])
    userRating[row['user_id']].append(row['rating'])

In [8]:
for index, row in us.iterrows():
    if int(row['anime_id']) in animeList:
        catList= getCategory(getData(animeList[int(row['anime_id'])]['genre']))
        for x in catList:
            userCategory[row['user_id']].add(x)
    

In [19]:
#Average rating per user including -1 for no review (KEEP or remove -1???)
averageUser=defaultdict(float)
stdUser=defaultdict(float)
for key in userRating:
    averageUser[key]=np.mean(userRating[key])
    stdUser[key]=np.std(userRating[key])